### Problem statement - We need to scrape the given URL and build two different data-sets for further analysis as follows:

#### First data-set should contain fields - `Sno`, `movie_name`, `director_name`, `duration, year`, `rating, metascore`.

#### Second data-set should contain fields -`movie_name`, `Stars` , `votes` , `genre` , `gross` , `popularity` , `certficiation`.

In [102]:
#Installing necessary libraries and upgrading versions - 

#pip install requests
#pip install --upgrade pip

In [103]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [104]:
#Getting the content from HTML-page and starting to parse the HTML-page:

url = "https://www.imdb.com/search/title/?genres=action&sort=user_rating,desc&title_type=feature&num_votes=25000,&pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=f11158cc-b50b-4c4d-b0a2-40b32863395b&pf_rd_r=XZ8X52H1R40B7KG5SNZ9&pf_rd_s=right-6&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_gnr_1"
response = requests.get(url)
# print(response)
soup = BeautifulSoup(response.content, 'html.parser')
# print(soup)

In [105]:
#Creating an empty list, so that we can append the values -

# CSv - 1:
Sno = []
movie_name = []
director_name = []
duration = []
year = []
rating = []
metascore = []

#CSV - 2:
Stars = []
votes = []
genre = []
gross = []
popularity = []
certifcation = []

In [106]:
#Storing the meaningfull required data in the variable
movie_data = soup.findAll('div', attrs= {'class': "lister-item mode-advanced"})
# <div class=>
# movie_name

In [107]:
#Calling molvie-name one-by-one using for loop and storing it in movie-name list ->

#For data-set -1 ->

for store in movie_data:
    name = store.h3.a.text
    movie_name.append(name)    
    
    serial = store.h3.find('span', class_ = "lister-item-index unbold text-primary").text.replace('.','')
    #print(serial)
    Sno.append(serial)

    runtime = store.p.find('span', class_ = 'runtime').text.replace(' min', '')
    duration.append(runtime)    
    
    #Scraping Director name -
    cast = store.find("p", class_ = '')
    cast = cast.text.replace('\n', '').split('|')
    cast = [x.strip() for x in cast]
    cast = [cast[i].replace(j, "") for i,j in enumerate(["Director:", "Stars:"])]
    director_name.append(cast[0])
    
    year_of_release = store.h3.find('span', class_ = "lister-item-year text-muted unbold").text.replace('(', '').replace(')', '')
    #print(year_of_release)
    year.append(year_of_release)
    
    rate = store.find('div', class_ = 'inline-block ratings-imdb-rating').text.replace('\n', '')
    rating.append(rate)
    
    meta  = store.find('span', class_ = 'metascore').text.replace(' ', '') if store.find('span', class_ = 'metascore') else '^^^^^^'
    metascore.append(meta)
    
#For data-set -2 ->

    #Cast Details -- Stars
    cast = store.find("p", class_ = '')
    cast = cast.text.replace('\n', '').split('|')
    cast = [x.strip() for x in cast]
    cast = [cast[i].replace(j, "") for i,j in enumerate(["Director:", "Stars:"])]
    Stars.append([x.strip() for x in cast[1].split(",")])    
    
    #Since, gross and votes have same attributes, that's why we had created a common variable and then used indexing
    value = store.find_all('span', attrs = {'name': 'nv'})    
    vote = value[0].text
    votes.append(vote)
    
    genre_list = store.p.find('span', class_ = "genre")
    #print("I am genre_list", genre_list)
    genre.append(genre_list)

    grosses = value[1].text if len(value) >1 else '*****'
    gross.append(grosses)
    
    #Popularity --
    rate = store.find('div', class_ = 'inline-block ratings-imdb-rating').text.replace('\n', '')
    popularity.append(rate)
    
    #Description of the Movies -- Not explained in the Video, But you will figure it out. 
    #describe = store.find_all('p', class_ = 'text-muted')
    #description_ = describe[1].text.replace('\n', '') if len(describe) >1 else '*****'
    #description.append(description_)
    
    #Certifcation --
    certificate_list = store.p.find('span', class_ = "certificate")
    #print("I am certificate_list", certificate_list)
    certifcation.append(certificate_list)

In [108]:
#Count the number of movies -

# np.count_nonzero(movie_name)
# sorted(year)
# sorted(Sno)
# sorted(rating)

In [109]:
#Creating a dataframe-1 using pandas library -

movie_df_1 = pd.DataFrame({'Serial Number': Sno, 'Name of movie': movie_name, "Director": director_name, 'Watchtime': duration, 'Year of relase': year, 'Movie Rating': rating, 'Metascore': metascore})
movie_df_1

,Serial Number,Name of movie,Director,Watchtime,Year of relase,Movie Rating,Metascore
0,1,The Dark Knight,Christopher Nolan,152,2008,9.0,84
1,2,The Lord of the Rings: The Return of the King,Peter Jackson,201,2003,9.0,94
2,3,Spider-Man: Across the Spider-Verse,"Directors:Joaquim Dos Santos, Kemp Powers, Jus...",140,2023,8.9,86
3,4,Inception,Christopher Nolan,148,2010,8.8,74
4,5,The Lord of the Rings: The Fellowship of the Ring,Peter Jackson,178,2001,8.8,92
5,6,The Lord of the Rings: The Two Towers,Peter Jackson,179,2002,8.8,87
6,7,The Matrix,"Directors:Lana Wachowski, Lilly Wachowski",136,1999,8.7,73
7,8,Star Wars: Episode V - The Empire Strikes Back,Irvin Kershner,124,1980,8.7,82
8,9,Soorarai Pottru,Sudha Kongara,153,2020,8.7,^^^^^^
9,10,Terminator 2: Judgment Day,James Cameron,137,1991,8.6,75


In [110]:
#Creating a dataframe-2 using pandas library -

movie_df_2 = pd.DataFrame({'Name of movie': movie_name, "Featuring Stars": Stars, 'Amount of votes': votes, 'Genre': genre, 'Gross': gross, 'Popularity': popularity, 'Certifcation': certifcation})
movie_df_2

,Name of movie,Featuring Stars,Amount of votes,Genre,Gross,Popularity,Certifcation
0,The Dark Knight,"[Christian Bale, Heath Ledger, Aaron Eckhart, ...","2,754,754","[\nAction, Crime, Drama ]",$534.86M,9.0,[UA]
1,The Lord of the Rings: The Return of the King,"[Elijah Wood, Viggo Mortensen, Ian McKellen, O...","1,904,438","[\nAction, Adventure, Drama ]",$377.85M,9.0,[U]
2,Spider-Man: Across the Spider-Verse,"[Shameik Moore, Hailee Steinfeld, Brian Tyree ...","198,554","[\nAnimation, Action, Adventure ]",*****,8.9,[U]
3,Inception,"[Leonardo DiCaprio, Joseph Gordon-Levitt, Elli...","2,445,450","[\nAction, Adventure, Sci-Fi ]",$292.58M,8.8,[UA]
4,The Lord of the Rings: The Fellowship of the Ring,"[Elijah Wood, Ian McKellen, Orlando Bloom, Sea...","1,932,702","[\nAction, Adventure, Drama ]",$315.54M,8.8,[U]
5,The Lord of the Rings: The Two Towers,"[Elijah Wood, Ian McKellen, Viggo Mortensen, O...","1,718,574","[\nAction, Adventure, Drama ]",$342.55M,8.8,[UA]
6,The Matrix,"[Keanu Reeves, Laurence Fishburne, Carrie-Anne...","1,976,913","[\nAction, Sci-Fi ]",$171.48M,8.7,[A]
7,Star Wars: Episode V - The Empire Strikes Back,"[Mark Hamill, Harrison Ford, Carrie Fisher, Bi...","1,333,514","[\nAction, Adventure, Fantasy ]",$290.48M,8.7,[UA]
8,Soorarai Pottru,"[Suriya, Paresh Rawal, Aparna Balamurali, Urva...","120,209","[\nAction, Drama ]",*****,8.7,[U]
9,Terminator 2: Judgment Day,"[Arnold Schwarzenegger, Linda Hamilton, Edward...","1,134,277","[\nAction, Sci-Fi ]",$204.84M,8.6,[A]


In [111]:
#Creating the two data-sets: 
movie_df_1.to_csv("first_dataset.csv")
movie_df_2.to_csv("second_dataset.csv")